<a href="https://colab.research.google.com/github/AnahitShekikyan/ADS-505-Final-Team-Project/blob/main/Final_Project_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Credit Card Fraud Detection Project**

## **Business Problem**
The objective of this project is to identify fraudulent credit card transactions. Credit card fraud detection is critical for financial institutions, as it helps prevent financial loss and maintain customer trust. We aim to build a model that can accurately detect fraudulent transactions, which account for only 0.17% of all transactions in our dataset.

## **Dataset Information**
The dataset contains 284,807 transactions, with 31 features describing each transaction. The target variable indicates whether the transaction is fraudulent (1) or legitimate (0). Given the class imbalance, specific techniques will be applied to address this challenge.
    

# **Import Data & Libraries**

In [ ]:
%%capture
!pip install dmba

#library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from dmba import classificationSummary
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score


# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Path to the dataset in Google Drive
data = '/content/drive/MyDrive/creditcard.csv'

# Load the CSV into a pandas DataFrame
df = pd.read_csv(data)

# Display the first few rows
df.head()

# **Basic Data Information**

In [ ]:
df.info()

In [ ]:
# Checking if there are there duplicates?
df.duplicated().sum()

In [ ]:
# Plotting the class distribution to visualize the class imbalance
plt.figure(figsize=(6, 4))
sns.countplot(x='Class', data=df)
plt.title('Distribution of Legitimate vs Fraudulent Transactions')
plt.show()

# Cheking if there is there class imbalance?
df['Class'].value_counts()

The histogram further confirms the extreme imbalance in the dataset with very few fraudulent instances compared to the legitimate ones. We need to handle the class imbalance and do dditional exploratory data analysis focusing on the minority class (fraud) to extract more relevant insights or patterns specific to those cases.


In [ ]:
# Plotting the transaction amounts for fraud vs non-fraud
plt.figure(figsize=(8, 6))
sns.boxplot(x='Class', y='Amount', data=df)

# Limiting y-axis to focus on smaller amounts for clearer visualization
plt.ylim(0, 500)
plt.title('Transaction Amounts by Class')
plt.show()

This box plot indicates that fraudulent transactions (Class 1) have higher median transaction amounts compared to legitimate transactions (Class 0). However, both classes show a wide range of transaction amounts, and there are more outliers in the legitimate transactions, so we need to use the "Amount" feature as a predictor since there appears to be a noticeable difference between classes. Investigate the outliers in the legitimate transactions to determine if they might be misclassified frauds or anomalies that need special handling, and normalize or standardize the "Amount" variable since it may vary significantly and could influence model performance if left unscaled.

# **Data Quality Report**


In [ ]:
# Summary stats
df.describe()

In [ ]:
def data_quality_report(df):

    # Initializing the report dictionary
    report = pd.DataFrame(index=df.columns)

    # Data types
    report['Data Type'] = df.dtypes

    # Counting missing values
    report['Missing Values'] = df.isnull().sum()

    # Counting percentage of missing values
    report['% Missing'] = (df.isnull().sum() / len(df)) * 100

    # Counting of unique values
    report['Unique Values'] = df.nunique()

    # Continuous features summary (only for float64 and int64)
    report['Min'] = df.min()
    report['Max'] = df.max()
    report['Mean'] = df.mean()
    report['Median'] = df.median()
    report['Standard Deviation'] = df.std()

    # Checking for duplicates
    report['Duplicates'] = df.duplicated().sum()

    # Determine cardinality for categorical variables (assumes non-continuous variables)
    report['Cardinality'] = [df[col].nunique() if df[col].dtype == 'object' else 'N/A' for col in df.columns]

    # Return the report
    return report

# Generating the Data Quality Report for the dataset
report = data_quality_report(df)

# Display the report
report

Here is a detailed summary of each feature in the dataset, covering various statistical and descriptive properties that are crucial for data exploration and preparation.


*   **Data Type:** All features, except the target variable (Class), are continuous and represented as floats.

*   **Missing Values:** There are no missing values, which indicates the dataset is complete and requires no imputation.


*   **Unique Values:** The number of distinct values in each feature. High cardinality (e.g., for Time and most other features) indicates a diverse set of values, typical for continuous variables.


*   **Min/Max:** It provides insight into the range of each feature. For example, the feature Amount ranges from 0 to 25691.16, which is consistent with transaction values.

*   **Mean/Median:**  This show that most features have means close to zero. This suggests that the features might have been transformed (e.g., PCA) to center their distributions.


*   **Standard Deviation:** It show the spread or dispersion of values within each feature. Features like "Amount" have a high standard deviation (250.12), reflecting a wide range of transaction values.

*   **Duplicates:** There are 1081 duplicate rows in the dataset, which may need to be removed or might prevent bias in model training.


*   **Cardinality:** Not applicable for continuous features.

Based on the data quality report we need to remove duplicates, handle outliers, scale the data, and  reduce multicollinearity.

# **Univariate Analysis**

**For Continuous Features**

In [ ]:
# Generating summary statistics for continuous variables
continuous_features = df.select_dtypes(include=['float64', 'int64']).columns
print("Summary statistics for continuous features:")
print(df[continuous_features].describe())

# Visualizing continuous variables in sets of 5 per row
n_features = len(continuous_features)
n_cols = 5  # Number of plots per row

for i in range(0, n_features, n_cols):
    plt.figure(figsize=(15, 4))  # Adjust the width for 3 plots per row
    for j in range(n_cols):
        if i + j < n_features:
            feature = continuous_features[i + j]

            # Create subplot for histogram and boxplot
            plt.subplot(1, n_cols, j + 1)
            sns.histplot(df[feature], bins=30, kde=True)
            plt.title(f'Distribution of {feature}')

    plt.tight_layout()  # Adjust layout for better spacing
    plt.show()

    # Second row with boxplots for the same features
    plt.figure(figsize=(15, 4))  # Separate figure for boxplots
    for j in range(n_cols):
        if i + j < n_features:
            feature = continuous_features[i + j]

            # Create subplot for boxplot
            plt.subplot(1, n_cols, j + 1)
            sns.boxplot(x=df[feature])
            plt.title(f'Boxplot of {feature}')

    plt.tight_layout()  # Adjust layout for better spacing
    plt.show()

There is a histograms and boxplots for each continuous feature to explore their distributions.

Histograms show the distribution of data within each feature. It helps identify skewness, modality (e.g., unimodal, bimodal), and if the data follows a normal distribution. For an example features like V1, V2, and others often show normal-like distributions centered around zero. This indicates they may have been scaled or transformed.

Boxplots display the spread and presence of outliers for each feature, and gave a view of the median, quartiles, and extreme values. For an exzamle of boxplots of features like V5, V6, etc., reveal many outliers, which can be important when deciding on preprocessing techniques, like applying robust scaling or addressing extreme values to avoid undue influence on the models.

Both histograms and boxplots are displayed side by side for each set of features, ensuring that the visual exploration of the dataset is comprehensive.

**For Categorical Features**

In [ ]:
# Visualizing the target variable 'Class'
plt.figure(figsize=(4, 3))
sns.countplot(x='Class', data=df)
plt.title('Distribution of Class (Fraud vs Non-Fraud)')
plt.show()

# Display percentage of fraud vs non-fraud transactions
class_counts = df['Class'].value_counts(normalize=True) * 100
print("Percentage distribution of the target variable 'Class':")
print(class_counts)

The bar chart displays the distribution of the target variable, Class, which represents legitimate transactions (Class 0) and fraudulent transactions (Class 1). It shows a significant imbalance between the two classes, with legitimate transactions making up 99.83% of the dataset, while fraudulent transactions only constitute 0.17%.

The extreme class imbalance indicated by the chart requires special consideration in the modeling process to ensure the model's effectiveness in identifying the minority class (fraudulent transactions). Proper resampling, algorithm adjustments, and appropriate evaluation metrics will be crucial to developing an effective fraud detection model.


# **Multivariate Analysis**

In [ ]:
# Using a subset of features to avoid too many plots (e.g., V1 to V5)
subset_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'Class']

# Creating a pair plot
sns.pairplot(df[subset_features], hue='Class', diag_kind='kde', plot_kws={'alpha': 0.3})
plt.show()

The pair plot shows the relationships and scatter distributions of selected features (e.g., V1, V2, etc.) across different classes. There are clusters and some separable patterns, particularly between legitimate and fraudulent classes. Here we need to identify and prioritize features that show clear separability between classes as important predictors for modeling. We might use the PCA if multicollinearity or highly correlated features are identified, and clustering algorithms to validate whether the clusters align with the fraud and non-fraud classes.

In [ ]:
# Computing the correlation matrix
corr_matrix = df.corr()

# Ploting the heatmap of the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

This heatmap illustrates the correlation between different features. Some features show strong correlations, which could indicate redundancy or multicollinearity issues. Here are the steps we might go for the next steps:


*   Removing or combine highly correlated features to avoid multicollinearity in model building (e.g., using PCA or dropping one of the correlated features).

*   Focusing on features with stronger correlations with the target variable (Class) as they might be more predictive.

In [ ]:
# Violin plot for 'Amount' based on 'Class'
plt.figure(figsize=(8, 6))
sns.violinplot(x='Class', y='Amount', data=df)
plt.title('Distribution of Transaction Amounts by Class')
plt.show()

The violin plot gives a deeper look into the distribution and density of transaction amounts by class. It shows that fraudulent transactions have a more concentrated distribution compared to legitimate ones, which are more spread out with outliers. Here we can explore additional features that might help differentiate frauds based on amount distributions, also handling outliers in the legitimate transactions to refine model performance.

In [ ]:
# Only keeping numeric features (skip target 'Class')
X = df.drop(columns=['Class'])

# Calculating VIF (Variance Inflation Factor) for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

The VIF output indicates the level of multicollinearity among the features in dataset.

Most features (e.g., "V1", "V3", "V6", "V8", and others) have VIF values close to 1, indicating very low multicollinearity. This suggests that these features are not significantly correlated with other features in the dataset, making them reliable for use in regression models.

Features like "V2", "V5", "V7", and "V20" have VIF values between 2 and 4. These values are still within an acceptable range but indicate some degree of correlation with other features. While these values do not warrant immediate removal, it's important to monitor these features in case they lead to multicollinearity issues in your model.

The "Amount" feature has a VIF value of approximately 11.5, which is quite high and indicates significant multicollinearity. This suggests that "Amount" may be highly correlated with one or more other features in the dataset. High VIF values like this could distort regression coefficients and affect the stability and interpretability of the model.

Since Amount has a high VIF value, you might need to explore its correlation with other features. If it's highly correlated with other variables, we can removing it, if it doesn't contribute new information or combine it with other features or transform it to reduce the correlation.

Although the VIF values between 2 and 4 are acceptable, it's a good idea to keep these features in mind when evaluating model performance, as they might introduce minor multicollinearity.

In [ ]:
# Group by 'Class' and compute summary statistics for continuous features
grouped_stats = df.groupby('Class').mean()
print(grouped_stats)

In [ ]:
# Box plot of Amount grouped by Class
plt.figure(figsize=(4, 3))
sns.boxplot(x='Class', y='Amount', data=df)
plt.title('Transaction Amount by Class')
plt.show()

This boxplot visualizes the distribution of transaction amounts for legitimate (Class 0) and fraudulent (Class 1) transactions. Fegitimate transactions (Class 0), the majority of transaction amounts are concentrated at lower values, with a few extreme outliers reaching up to over 25,000 and but  presence of numerous outliers suggests that legitimate transactions vary widely in amount, with most being small but some being very large.

For fraudlent transactions (Class 1) show low fraudulent transaction but appear to be more tightly distributed, with significantly fewer extreme outliers compared to legitimate transactions. Which suggests that fraudulent transactions generally tend to have lower amounts, and there is less variation in their values.

# **Data Preprocessing**
In this section, we will handle missing values, scale the data, and address class imbalance.

In [ ]:
# Removing duplicates
df = df.drop_duplicates()

In [ ]:
# Checking if there are there still any duplicates?
df.duplicated().sum()

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()

# Apply scaling (excluding the target 'Class')
df_scaled = df.copy()
df_scaled.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])

# Verify scaling
df_scaled.head()

The output indicates that the values are scaled between 0 and 1, confirming that MinMaxScaler was applied correctly.

Our dataset contains substantial outliers, which is common in fraud detection, so RobustScaler is primary choice, because this scaler centers the data using the median and scales using the interquartile range (IQR), which makes it robust to outliers. So, to ensure that outliers are minimized first we will use  RobustScaler and then further normalize the data for algorithms sensitive to scaling StandardScaler.




In [ ]:
# Applying RobustScaler first
robust_scaler = RobustScaler()
X_robust_scaled = robust_scaler.fit_transform(X)

# Applying StandardScaler on the robust-scaled data
standard_scaler = StandardScaler()
X_final_scaled = standard_scaler.fit_transform(X_robust_scaled)

Let's create a visualization technique which will use boxplots to compare the distribution of features (V17, V14, V12, V10) between the two classes (0 for non-fraud and 1 for fraud). It's particularly useful for understanding how these features correlate with the target class.

In [ ]:
# Assuming that df is our original DataFrame
columns_needed = ['Class', 'V17', 'V14', 'V12', 'V10']

# Create a new DataFrame with the selected columns
new_df = df[columns_needed]

# Setting the color palette
colors = ['#1f77b4', '#ff0000']

# Creating a figure with 4 subplots in one row
f, axes = plt.subplots(ncols=4, figsize=(20, 4))

# Plotting boxplot for each feature against Class
sns.boxplot(x="Class", y="V17", data=new_df, palette=colors, ax=axes[0])
axes[0].set_title('V17 vs Class Negative Correlation')

sns.boxplot(x="Class", y="V14", data=new_df, palette=colors, ax=axes[1])
axes[1].set_title('V14 vs Class Negative Correlation')

sns.boxplot(x="Class", y="V12", data=new_df, palette=colors, ax=axes[2])
axes[2].set_title('V12 vs Class Negative Correlation')

sns.boxplot(x="Class", y="V10", data=new_df, palette=colors, ax=axes[3])
axes[3].set_title('V10 vs Class Negative Correlation')

# Display the plot
plt.tight_layout()
plt.show()

The variables displayed in this plot (e.g., V17, V14, V12, and V10) show strong negative correlations with the target class. This indicates that as the values of these variables increase, the likelihood of fraud decreases.

We need to:

*   Incorporate these variables prominently in the model as they demonstrate strong predictive potential.

*   Use feature engineering to create interaction terms or non-linear transformations if these variables exhibit non-linear relationships with the target.


In [ ]:
# Creating a correlation matrix with imbalanced and SubSample data
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0].sample(n=len(fraud), random_state=42)

# Concatenate fraud and non-fraud samples to create a balanced subsample
subsample = pd.concat([fraud, non_fraud])

# Compute correlation matrices
corr_matrix_full = df.corr()
corr_matrix_subsample = subsample.corr()

# Set up the matplotlib figure
fig, ax = plt.subplots(2, 1, figsize=(8, 6))

# Imbalanced correlation matrix
sns.heatmap(corr_matrix_full, cmap='coolwarm', ax=ax[0], cbar_kws={'shrink': 0.5}, vmin=-1, vmax=1)
ax[0].set_title('Imbalanced Correlation Matrix\n(don\'t use for reference)', fontsize=16)

# Subsample correlation matrix
sns.heatmap(corr_matrix_subsample, cmap='coolwarm', ax=ax[1], cbar_kws={'shrink': 0.5}, vmin=-1, vmax=1)
ax[1].set_title('SubSample Correlation Matrix\n(use for reference)', fontsize=6)

plt.tight_layout()
plt.show()

The two correlation matrices illustrate the relationship between different features in the dataset, showing how preprocessing, especially subsampling, impacts the representation and correlations between features.

Imbalanced correlaition matrix show the  lacks variability and detail, indicating that the class imbalance masks true relationships between features. This is because the overwhelming majority of non-fraudulent (Class 0) data points dominate the calculation, making the correlation values less reliable for model development. This matrix is not ok to  be used as a reference for feature selection or understanding relationships since it is skewed by the imbalance.

SubSample correlation matrix shows a more diverse pattern of correlations, with a mixture of positive and negative correlations across different feature pairs. This suggests that the subsampled dataset is better balanced, allowing the true relationships to surface. There is a
strong correlations between some features (e.g., between V3, V6, and V9) which could indicate multicollinearity. These features might need to be addressed using techniques like Principal Component Analysis (PCA) or feature elimination to reduce redundancy.



In [ ]:
# Assumeing df is the dataset and features are the continuous features that we apply PCA on
features = df.select_dtypes(include=['float64', 'int64']).columns
X = df[features]

# First standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Now applying PCA
pca = PCA(n_components=0.95)  # adjusting the "n_components" parameter in PCA accordingly.
X_pca = pca.fit_transform(X_scaled)

# Visualizing the explained variance
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by Components')
plt.show()

# Transforming the data
# The transformed dataset (X_pca) now has reduced dimensions based on the number of components chosen
pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

# Adding the target variable back to the transformed dataset if needed
pca_df['Class'] = df['Class']

# Displaying the first few rows of the PCA-transformed dataset
print(pca_df.head())



The Principal Component Analysis (PCA) plot illustrates how the cumulative variance is explained as more principal components are added. In this plot, the variance steadily increases with each additional component, eventually reaching close to 100% when all components are included. This indicates that PCA effectively captures the variance present in the original dataset, allowing us to reduce the dataset's dimensionality while retaining most of the important information. We select the number of components that capture around 95% of the variance, which strikes a balance between dimensionality reduction and preserving the dataset's integrity.

By examining the PCA output, it's clear that the principal components (e.g., PC1, PC2, etc.) are new, uncorrelated dimensions that summarize the original features. These components can be used for further model training, reducing noise and multicollinearity. The explained variance plot guides us in determining the optimal number of components to retain, ensuring that we maintain a high proportion of the dataset’s variance while improving model efficiency and performance. This dimensionality reduction process is crucial for simplifying the dataset, making it more manageable and suitable for building predictive models.


# Model 1 _ Logistic Regression

In [ ]:
# Assuming 'X_pca' is the PCA-transformed feature set and 'y' is the target variable, df is the original dataset
y = df['Class']

# Spliting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Initializing the logistic regression model, adjusting max_iter if necessary for convergence
log_reg = LogisticRegression(random_state=42, max_iter=1000)

# Training the model using the training data
log_reg.fit(X_train, y_train)

# Making predictions on the test set
y_pred = log_reg.predict(X_test)
y_pred_prob = log_reg.predict_proba(X_test)[:, 1]

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Calculating AUC
auc = roc_auc_score(y_test, y_pred_prob)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Displaying the evaluation metrics
print("Logistic Regression Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)
print("\nAUC Score:", auc)

#Model 2 _ Random Forest Classifier

In [ ]:
# Initialize RandomForestClassifier with class weights to handle imbalance
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Fitting the model
rf_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = rf_model.predict(X_test)

# Displaying the evaluation metrics
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Access one of the trees in the forest (e.g., the first tree)
tree = rf_model.estimators_[0]  # Accessing the first tree in the forest

# Visualizing the tree
plt.figure(figsize=(10, 8))
plot_tree(tree,
          filled=True,
          feature_names=X_train.columns if isinstance(X_train, pd.DataFrame) else ['Feature_' + str(i) for i in range(X_train.shape[1])],
          class_names=['Not Fraud', 'Fraud'],
          rounded=True)
plt.title('Decision Tree Visualization (Tree 1 of Random Forest)')
plt.show()


The decision tree visualization provided represents one of the individual trees from the Random Forest model built to classify transactions as either fraudulent or non-fraudulent. The tree is composed of several levels where splits are made based on specific feature values, with each node representing a decision point. The tree uses these splits to divide the dataset into smaller subsets, attempting to group similar observations together. The decision rules at each node are determined based on features that maximize information gain, measured by a reduction in Gini impurity, which indicates how mixed the classes are within a node. A Gini value of 0 at a node means that all samples belong to a single class, making the node pure.

As the tree progresses down each path, it uses different features, such as Feature_1, Feature_14, and Feature_12, suggesting that these attributes are influential in distinguishing between fraudulent and non-fraudulent transactions. Early splits in the tree rely on these critical features, helping the model make initial decisions about which direction the transaction should proceed. The tree then continues to branch out further using other features, each split further refining the classification until the leaf nodes are reached. These leaf nodes represent the final classification for the observations, where the "class" value shows whether the majority of samples at that node are classified as fraud or not fraud. It’s notable that some branches become pure (Gini = 0) quite quickly, indicating a clear separation of classes based on certain feature values, while other branches require more depth to achieve purity.

This individual tree is just one part of the entire Random Forest, which is composed of many such trees, each capturing different aspects of the data. While this single tree provides insight into how certain features and their values influence the classification process, it is important to remember that the forest, as a whole, averages the predictions from all these trees, reducing the risk of overfitting. Overfitting is a concern when trees grow too deep and fit noise in the training data rather than capturing general patterns. However, the Random Forest model mitigates this by aggregating results across multiple trees, leading to a more robust and generalizable model. This visualization allows us to understand the decision-making process within the forest and highlights the importance of certain features in classifying fraudulent versus non-fraudulent transactions.

#Model 3 _ SVM Model

In [ ]:
# Assuming that X_scaled is the scaled feature set and y is the target variable
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initializing the SVM model with class weights to handle class imbalance
svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)

# Training the model using the training data
svm_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = svm_model.predict(X_test)

# Getting prediction probabilities for AUC calculation
y_pred_prob = svm_model.predict_proba(X_test)[:, 1]

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Calculating AUC
auc = roc_auc_score(y_test, y_pred_prob)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Displaying the evaluation metrics
print("SVM Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)
print("\nAUC Score:", auc)

#Model 4 _ XGBOOST

In [ ]:
# Assuming 'X' and 'y' are your scaled and processed features and target variable
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing XGBoost with scale_pos_weight to handle class imbalance
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, scale_pos_weight=scale_pos_weight)

# Fitting the model to the training data
xgb_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = xgb_model.predict(X_test)
y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Calculating AUC
auc = roc_auc_score(y_test, y_pred_prob)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Displaying the evaluation metrics
print("XGBoost Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)
print("\nAUC Score:", auc)

#ROC Curves for All The Models

In [ ]:
# Initializing the models
log_reg = LogisticRegression(random_state=42, max_iter=1000)
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
svm_model = SVC(probability=True, random_state=42)

# Fitting models on training data and get probabilities for ROC
models = {
    'Logistic Regression': log_reg,
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'SVM': svm_model
}

plt.figure(figsize=(8, 6))

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    auc = roc_auc_score(y_test, y_pred_prob)
    plt.plot(fpr, tpr, label=f"{model_name} (AUC = {auc:.2f})")

plt.plot([0, 1], [0, 1], linestyle='--', color='black')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Multiple Models')
plt.legend()
plt.show()

The ROC curve visualization compares the performance of four models.

**Logistic Regression (AUC = 0.98):** This model performs very well, with an AUC (Area Under the Curve) of 0.98. The ROC curve is close to the top-left corner, indicating that the model has a high true positive rate and a low false positive rate. It is a near-optimal model for this dataset.

**Random Forest (AUC = 1.00):** The Random Forest model achieves a perfect AUC of 1.00, which indicates separation between classes. The ROC curve follows the axes precisely, meaning it correctly identifies all true positives and negatives without any error. This model provides the best fit among all.

**XGBoost (AUC = 1.00):** Similar to Random Forest, the XGBoost model also achieves a perfect AUC of 1.00. The curve again touches the top-left corner, demonstrating that the model perfectly distinguishes between fraudulent and non-fraudulent cases. Both Random Forest and XGBoost models perform exceptionally well and are likely the best choices for this dataset.

**SVM (AUC = 0.45):** The SVM model, with an AUC of 0.45, performs poorly compared to the others. Its ROC curve is below the diagonal line, indicating that it performs worse than random guessing. This suggests that the SVM model is not suitable for this particular dataset or problem without further tuning or feature engineering.

#Evaluating The Models

In [ ]:
# Initializing an empty list to store results
results = []

# Training and evaluating each model
for model_name, model in models.items():
    # Fitting the model
    model.fit(X_train, y_train)

    # Making predictions
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probability for the positive class (Fraud)

    # Calculating metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)

    # Append results to the list
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': auc
    })

# Converting results to DataFrame
df_results = pd.DataFrame(results)

# Displaying the results
display(df_results)